# Import backage

In [1]:
import sys
sys.path.append("../..")
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import random
%matplotlib inline

In [2]:
import torch
from torch import nn
from torch import distributions
from torch.nn.parameter import Parameter
from train import train
from models.models_new import Renorm_Dynamic
from dynamic_models_sis_new import Simple_Spring_Model
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KernelDensity
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from datetime import datetime
t0 = datetime.now()


use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0') if use_cuda else torch.device('cpu')
device

device(type='cuda', index=0)

# Noise experiments

In [3]:
# nis+ CE macro
mul_batch_size = [0,5000,3000,1000]
sigma_list = [0.00001,0.0001,0.001,0.01,0.1,1]
experiments = len(sigma_list)
rou = -0.5
steps = 7
sz = 4
scale = 2
L = 1
mae2_w = 3
T_total = 30001


eis_macro_e = np.zeros([experiments,T_total//500+1])
losses_macro_e = np.zeros([experiments,T_total//500+1])
MAEs_mstep_macro_e = np.zeros([experiments,T_total//500+1])

In [4]:

for e in range(experiments):
    sigma = sigma_list[e]
    seed = 1 
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    spring = Simple_Spring_Model(device=device)
    test_data = spring.generate_multistep_sir(size_list=[500,500], steps=10, sigma=sigma, rou=rou,lam=1,miu=0.5,dt=0.01) #sir
    train_data = spring.generate_multistep_sir(size_list=mul_batch_size, steps=steps, sigma=sigma,rou=rou,lam=1,miu=0.5,dt=0.01)
    
    eis_nisp, term1s_nisp, term2s_nisp, losses_nisp, MAEs_mstep_nisp, net = train(train_data=train_data, test_data=test_data, 
                                                                    sigma=sigma, rou=rou, sz=sz, scale=scale, L=L, 
                                                                    mae2_w=mae2_w, T2=T_total,framework='nis+')
    eis_macro_e[e,:] = eis_nisp
    losses_macro_e[e,:] = losses_nisp
    MAEs_mstep_macro_e[e,:] = MAEs_mstep_nisp
    torch.save(net.state_dict(), 'mdl_data/macro noise = '+str(sigma)+'.mdl')

check point-------o_0-> 11:09:19.901573; lasting 0:00:02.575549 seconds
Epoch: 0
Train loss: 0.5386
dEI: 0.2611
term1: -0.7615
term2: 1.2837
Test multistep loss: 0.6087
------------------------------------------------------------------------------------------------------------------------
check point-------o_1-> 11:09:24.529557; lasting 0:00:04.627984 seconds
check point-------o_0-> 11:09:35.752731; lasting 0:00:11.223174 seconds
Epoch: 500
Train loss: 0.0567
dEI: 2.2358
term1: 2.8689
term2: 1.6026
Test multistep loss: 0.0651
------------------------------------------------------------------------------------------------------------------------
check point-------o_1-> 11:09:40.609817; lasting 0:00:04.857086 seconds
check point-------w_0-> 11:09:51.484529; lasting 0:00:10.874712 seconds
check point-------w_1-> 11:09:51.502873; lasting 0:00:00.018344 seconds
check point-------w_2-> 11:14:41.647047; lasting 0:04:50.144174 seconds
check point-------w_3-> 11:14:41.676806; lasting 0:00:00.02

In [6]:
# nis+ micro
scale = 4
eis_micro_e = np.zeros([experiments,T_total//500+1])
losses_micro_e = np.zeros([experiments,T_total//500+1])
MAEs_mstep_micro_e = np.zeros([experiments,T_total//500+1])

for e in range(experiments):
    sigma = sigma_list[e]
    seed = 1 
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    spring = Simple_Spring_Model(device=device)
    test_data = spring.generate_multistep_sir(size_list=[500,500], steps=10, sigma=sigma, rou=rou,lam=1,miu=0.5,dt=0.01) #sir
    train_data = spring.generate_multistep_sir(size_list=mul_batch_size, steps=steps, sigma=sigma,rou=rou,lam=1,miu=0.5,dt=0.01)
    
    eis_nisp, term1s_nisp, term2s_nisp, losses_nisp, MAEs_mstep_nisp, net = train(train_data=train_data, test_data=test_data, 
                                                                    sigma=sigma, rou=rou, sz=sz, scale=scale, L=L, 
                                                                    mae2_w=mae2_w, T2=T_total,framework='nis+')
    eis_micro_e[e,:] = eis_nisp
    losses_micro_e[e,:] = losses_nisp
    MAEs_mstep_micro_e[e,:] = MAEs_mstep_nisp
    torch.save(net.state_dict(), 'mdl_data/micro noise = '+str(sigma)+'.mdl')

check point-------o_0-> 10:00:54.040520; lasting 3:45:12.542183 seconds
Epoch: 0


RuntimeError: The size of tensor a (4) must match the size of tensor b (2) at non-singleton dimension 1

In [ ]:
plt.figure(dpi=300)
color_label = ['#F4F1DE', '#DF7A5E', '#3C405B', '#82B29A', '#F2CC8E', '#A8DADC', '#457B9D', '#1D3557']
plt.plot(sigma_list, eis_macro_e[:,-1]-eis_micro_e[:,-1])
plt.legend()
plt.xlabel("epoch")
plt.ylabel(r'$\mathcal{J}$')
plt.show()

# Psi calculate

In [ ]:
import ei.entropy_estimators as ee

def psi(net,micro_data):
    ss,sps,ls,lps = micro_data
    predicts1, latent1, latentp1 = net.forward(ss)
    micro_mi = 0
    for dim in range(ss.size()[1]):
        micro_mi += ee.mi(ss[:,dim].cpu().numpy(),latenp1.cpu().numpy())
    macro_mi = ee.mi(latent1.cpu().numpy(), latentp1.cpu().numpy())
    p = macro_mi - micro_mi
    return p